# Data Warehouse

## Introduction
- Data warehousing is a technique used to consolidate data from multiple sources into a single, centralized repository for faster querying, analysis, and reporting
- Example: In large businesses that deal with frequent transactions on different servers, data is often distributed across multiple systems and databases, making it difficult to query and analyze in real-time. By consolidating this data into a data warehouse, businesses can create a single source of truth for all their data, making it easier to access, analyze, and report on.
- typically a data warehouse:
    - its optmized for data querying (read operations)
    - incrementaly loaded with new data at regular intervals (write operations)

## Data Warehouse Architectures

- __centralized data warehouse:__ 
    - centralized data warehouse is a large storage that contains multiple databases, each of which corresponds to a specific subject area or a department of an organization. Each database contains strictly defined tables and columns that hold all the historical data needed for analysis.
    - users pull data directly from the warehouse for analysis
- __hub-and-spoke:__
   - is a centralized data warehouse complimented by data marts
   - users pull data from their respective data mart at much higher speed
    - __data mart:__
       - The process of creating a data mart involves selecting and summarizing the data from the larger data warehouse in a way that is relevant and useful to a specific business unit or department. The data is then transformed into a format that is optimized for fast querying and analysis, and loaded into a separate physical storage.
       - provide faster query time (less computation expences) but duplicates the data so requires additional space (more storage expenses)

## Components Of A Data Warehousing Solution
- data sources > staging etl > staging database > etl > data warehouse

## Normalization
- De-normalizing data 
    - improve data read and write performance by reducing the need for complex joins across multiple tables. 
    - requires duplicating data, which can increase storage requirements and potentially lead to data consistency issues.
    - commonly used for transactional systems, where query performance is critical and data consistency can be maintained through other means, such as transactional control mechanisms


- Normalizing data
    - help to minimize data redundancy and improve data consistency
    - it may increase the complexity of queries and slow down data read and write performance, especially when dealing with complex queries across multiple tables.
    -  preferred for data warehousing and reporting purposes, where data consistency and accuracy are critical and performance is less of a concern

## Fact Tables
- A fact table is a table in a data warehouse that stores the quantitative data or measurements of a business process or event. It typically contains the foreign keys that link to related dimension tables, as well as the numeric values that are used to perform calculations, such as sums, averages, and other statistical functions.


- For example, in the a retail store where there are an order and product tables (for simplicity each order have one product only):
    1. the fact table can be interpreted as a join between the order and product tables
    2. the columns selected include foreign key from the order and product table (dimension tables primary keys)
    3. the columns selected also include key numeric metrics like quantity and price
    4. the fact table can provide complex analysis by aggregating the numeric columns without multiple complex joins
        - ex: calculating the total revenue for a specific product category

            ```sql
            SELECT SUM(sales) 
            FROM  fact_table
            WHERE product_category = 'product' 
            -- a join with the product table was avoided thus a faster query
            ----------------------------------------------------------------
            ```

    5. the fact table can be joined with a dimension table to provide additional information a
        - ex: identifying the top-selling products for a particular customer segment

            ```sql
            SELECT f.product,c.customer_segment, count(f.sales) as cnt
            FROM  fact_table f
            JOIN customer c ON c.id = f.id
            GROUP BY f.product
            HAVING c.customer_segment = 'segment'
            ORDER BY cnt DESC
            -- this query would have required multiple joins but the fact table reduced the number of joins to 1
            ------------------------------------------------------------------------
            ```

- Usually its computationally expensive to query the fact table like results through joins, so a fact table is defined in the warehouse schema taking more storage space but saving computational resources as a form of de-normalization.

- a data warehouse should have one truth table for every process:
    - if two databases are related to the same process the truth table should be a join between the dimension tables of the two databases

## Data Warehouse Schemas

### Star Schema

- The star schema is designed to optimize query performance and provide simplicity.
    - it results in larger disk space, however, quires faster and less complex.
- the dimension tables are not normalized.
- __ALL__ dimension tables are connected to the fact tables 
    - no-suitable only one join needed to connect to another table 
- __example:__ the fact table is "Sales" and contains the quantitative data related to each sale, such as the date, product ID, and sales amount. The dimension tables are "Products", "Customers", and "Time", and each contains descriptive information about the sale, such as the product name, customer name, and sale date.
```sql
SELECT p.category, SUM(s.sales_amount) as total_sales
FROM Sales s
JOIN Products p ON s.product_id = p.product_id
GROUP BY p.category
```
- __Visual Example:__
    - ![star](https://streamsets.b-cdn.net/wp-content/uploads/star-schema.png)

### Snowflake Schema

- The snowflake schema is designed to reduce data redundancy and improve data integrity
    - it can result in slower query performance due to the increased number of joins required.
- the dimension tables are normalized into multiple related tables.
- the truth table is connected to __SOME__ but not all of the normalized dimension tables.
    - sub tables are present and some tables require multiple joins to connect to a normalized table
- __Example__: the fact table is "Sales" and contains the quantitative data related to each sale, such as the date, product ID, and sales amount. The "Products" dimension table is normalized into multiple related tables, including "Product", "Product Category", and "Manufacturer". The "Customers" and "Time" dimension tables are the same as in the star schema example.
```sql
SELECT pc.category, SUM(s.sales_amount) as total_sales
FROM Sales s
JOIN Product p ON s.product_id = p.product_id
JOIN Product_Category pc ON p.category_id = pc.category_id
GROUP BY pc.category
```
- __Visual Example:__
    - ![snowflake](https://streamsets.b-cdn.net/wp-content/uploads/Snowflake-Schema.png)

## Distributed Vs Non-Distributed

- Non-distributed Data Warehouse Solutions:
    - Run on a single server or small cluster
    - Easier to manage and maintain
    - Limited scalability, fault tolerance, and processing speed

- Distributed Data Warehouse Solutions:
    - Run on a large cluster of servers
    - Data is distributed for better fault tolerance and redundancy
    - Greater scalability, processing speed, and complexity
    - Higher cost due to more hardware and infrastructure

- Metrics for system cno:
    - __Storage capacity:__ Understand the amount of storage capacity needed to store the data warehouse. This will depend on the size of the data warehouse, the amount of data to be stored, and the growth rate of the data.

    - __Performance requirements:__ Understand the performance requirements of the data warehouse, including the query response time, data load time, and system availability. This will help you determine the appropriate storage solution to meet these requirements.

    - __Storage options:__ Learn about the different storage options available for data warehousing, including hard disk drives (HDDs), solid-state drives (SSDs), cloud-based storage, and distributed storage solutions such as storage area networks (SANs) and network-attached storage (NAS).

    - __Cost:__ Understand the cost implications of different storage solutions and how they impact the overall cost of the data warehouse.

    - __Data redundancy and disaster recovery:__ Learn about the importance of data redundancy and disaster recovery for data warehousing and how different storage solutions can impact these factors.